In [99]:
import os
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path

from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

In [77]:
load_dotenv()

True

In [78]:
DIMENSION = 1536
COLLECTION_NAME = "driving"

In [79]:
connections.connect(host=os.environ.get("MILVUS_HOST"), port=os.environ.get("MILVUS_PORT"))

In [80]:
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

In [81]:
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="town", dtype=DataType.VARCHAR, max_length=16),
    FieldSchema(name="section", dtype=DataType.INT64),
    FieldSchema(name="scenario", dtype=DataType.VARCHAR, max_length=48),
    FieldSchema(name="part", dtype=DataType.INT64),
    FieldSchema(name="objects", dtype=DataType.VARCHAR, max_length=4096),
    FieldSchema(name="controls", dtype=DataType.VARCHAR, max_length=4096),
    FieldSchema(name="location", dtype=DataType.VARCHAR, max_length=128),
    FieldSchema(name="situation", dtype=DataType.VARCHAR, max_length=1024),
    FieldSchema(name="risk", dtype=DataType.VARCHAR, max_length=16),
    FieldSchema(name="message", dtype=DataType.VARCHAR, max_length=4096),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]

schema = CollectionSchema(fields)

driving_collection = Collection(COLLECTION_NAME, schema, consistency_level="Strong")

In [82]:
index_params = {
    'index_type': 'FLAT',
    'metric_type': 'L2',
    'params': {'nlist': 1024}
}
driving_collection.create_index(field_name="embedding", index_params=index_params)

Status(code=0, message=)

In [83]:
df = pd.read_csv(Path("../data/training.csv"))
df.head(5)

,town,section,scenario,part,objects,controls,chatgpt_raw_response,chatgpt_extracted_location,chatgpt_extracted_situation,chatgpt_extracted_risk,chatgpt_extracted_message,objs_ctrls_embedding
0,Town12,81,StaticCutIn,1,"[(30, 'vehicle.ford.mustang'), (15, 'vehicle.m...","[{'throttle': 0.75, 'steer': 2.133598718501161...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching multiple vehicles,medium,"Stay focused, be aware of surrounding vehicles...","[-0.034090910106897354, 0.0011322549544274807,..."
1,Town12,81,StaticCutIn,1,"[(29, 'vehicle.ford.mustang'), (14, 'vehicle.m...","[{'throttle': 0.75, 'steer': 0.000891609524842...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching multiple vehicles and a traffic light,medium,"Stay alert, approaching vehicles and traffic l...","[-0.03626420348882675, 0.000824744172859937, 0..."
2,Town12,81,StaticCutIn,1,"[(28, 'vehicle.ford.mustang'), (13, 'vehicle.m...","[{'throttle': 0.75, 'steer': -0.00017667813517...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching multiple vehicles and a traffic light,medium,"Stay focused, be prepared to slow down for the...","[-0.037175726145505905, -0.0005778020131401718..."
3,Town12,81,StaticCutIn,1,"[(30, 'vehicle.nissan.patrol_2021'), (30, 'veh...","[{'throttle': 0.0, 'steer': 0.0002305144735146...","{\n ""location"": ""Urban area"",\n ""situation"":...",Urban area,Approaching multiple vehicles and a stop sign,medium,"Stay alert, slow down, and prepare to stop at ...","[-0.03076704777777195, 0.0018085219198837876, ..."
4,Town12,81,StaticCutIn,1,"[(48, 'vehicle.mini.cooper_s_2021'), (7, 'vehi...","[{'throttle': 0.0, 'steer': -0.000179785565705...","{\n ""location"": ""City street"",\n ""situation""...",City street,Approaching a stop sign with vehicles nearby,medium,"Approach the stop sign cautiously, there are v...","[-0.03350893780589104, -0.009793606586754322, ..."


In [90]:
for index, row in df.iterrows():
    entities = row.to_dict()
    entities["embedding"] = [float(x) for x in entities["objs_ctrls_embedding"].replace('[', '').replace(']', '').split(",")]
    entities["location"] = entities.pop("chatgpt_extracted_location")
    entities["situation"] = entities.pop("chatgpt_extracted_situation")
    entities["risk"] = entities.pop("chatgpt_extracted_risk")
    entities["message"] = entities.pop("chatgpt_extracted_message")
    entities.pop("objs_ctrls_embedding")
    entities.pop("chatgpt_raw_response")
    driving_collection.insert(entities)

# For checking if the data is inserted correctly

In [85]:
driving_collection.load()

In [86]:
test_query = [-0.034090910106897354, 0.0011322549544274807, 0.012882701121270657, -0.017200414091348648, -0.029075883328914642, 0.01101615373045206, -0.006184260360896587, 0.011952949687838554, -0.02289162389934063, -0.04871337488293648, -0.012833395972847939, 0.04598046839237213, -0.0013101052027195692, -0.0015742393443360925, 0.0014166393084451556, 0.020834898576140404, 0.005941256880760193, -0.038796018809080124, 0.007036532741039991, -0.015171864069998264, 0.006575178820639849, 0.00907212682068348, -0.01596074365079403, -0.009586307220160961, -0.041585274040699005, 0.008198723196983337, 0.007395755499601364, -0.02244083397090435, 0.003947924822568893, -0.039556726813316345, 0.020581331104040146, -0.03132982924580574, -0.018045643344521523, -0.03516153246164322, -0.013298272155225277, 0.005698253400623798, 0.010177968069911003, -0.0270755086094141, 0.02631480246782303, -0.002505752258002758, -0.00027580003370530903, 0.013439144007861614, -0.0077620213851332664, -0.012903831899166107, -0.003148478688672185, 0.02221544086933136, 0.017270850017666817, 0.006585744209587574, -0.027568558230996132, 0.021454734727740288, 0.006346262525767088, 0.02876596711575985, -0.026483846828341484, -0.014242111705243587, 0.02446938492357731, 0.006240608636289835, -0.003845792729407549, 0.02458208240568638, -0.025286439806222916, -0.020355936139822006, 0.014974643476307392, 0.0033597860019654036, -0.029949286952614784, -0.004891763906925917, -0.00907212682068348, 0.008417073637247086, 0.011847295798361301, 0.00388805428519845, 0.02368050441145897, 0.025849925354123116, 0.018285125494003296, -0.0015407823957502842, 0.01743989624083042, 0.007036532741039991, -0.0012880940921604633, -0.026188017800450325, -0.016115702688694, 0.007804282940924168, 0.008973516523838043, -0.0018454170785844326, 0.020722202956676483, -0.018623216077685356, -0.040458302944898605, 0.04364199936389923, 0.016059353947639465, -0.006120868027210236, -0.008043764159083366, 0.012770004570484161, 0.016848234459757805, -0.01791885867714882, -0.009980747476220131, 0.02070811577141285, 0.010678062215447426, 0.0053848144598305225, -0.004430409520864487, 0.019482532516121864, -0.031498875468969345, 0.01712997816503048, -0.004014838486909866, -0.00917073618620634, 0.0031942620407789946, -0.011206329800188541, -0.024722954258322716, -0.012234692461788654, -0.019834712147712708, -0.03211870789527893, 0.013861758634448051, 0.007804282940924168, 0.007585932034999132, -0.007395755499601364, -0.0023666417691856623, 0.028709618374705315, 0.0009693722240626812, -0.02782212756574154, -0.01500281784683466, 0.02330015040934086, -0.007430972997099161, 0.007388711906969547, -0.01312218327075243, -0.02556818351149559, 0.0008989364723674953, 0.04733283445239067, 0.0022486618254333735, -0.034964315593242645, 0.01705954223871231, -0.010325883515179157, 0.009846920147538185, -0.016510143876075745, -0.016777798533439636, -0.018496431410312653, 0.027314988896250725, 0.017186326906085014, 0.013002442196011543, -0.0006471286178566515, 0.0028015824500471354, 0.018130166456103325, -0.008212810382246971, 0.01370679959654808, -0.02251126989722252, -0.018242863938212395, -0.0061032590456306934, 0.027892563492059708, -0.010671018622815609, 0.023736853152513504, -0.00794515386223793, 0.0017115890514105558, 0.0058320811949670315, 0.005455249920487404, -0.0008914526551961899, -0.019031744450330734, 0.005141810979694128, 0.004740327131003141, 0.007170361001044512, 0.009938486851751804, -0.004078230820596218, 0.003173131262883544, 0.026159843429923058, -0.008522727526724339, 0.014749249443411827, 0.004405756946653128, 0.014523854479193687, 0.031245306134223938, -0.00035393971484154463, -0.0345698744058609, 0.00895942933857441, 0.015876220539212227, 0.012431912124156952, 0.011072502471506596, -0.02523009106516838, -0.011459899134933949, -0.024455297738313675, 0.03513335809111595, -0.01108658965677023, -0.018665477633476257, -0.012953137047588825, 0.03209053352475166, 0.025737227872014046, 0.013657494448125362, -0.046628475189208984, 0.0036873123608529568, 0.0135166235268116, 0.022004133090376854, 0.03192148730158806, 0.049417734146118164, 0.010438580065965652, -0.03499248996376991, -0.012403738684952259, -0.03614763543009758, 0.007311232388019562, -0.0007237275131046772, 0.03941585496068001, 0.03166792169213295, -0.003115021623671055, -0.008593163453042507, -0.6216378808021545, -0.01532682217657566, 0.016538318246603012, -0.02804752252995968, 0.004691021982580423, 0.04431818425655365, 0.006937922909855843, -0.010107532143592834, -0.03862697258591652, 0.03572501987218857, -0.013622277416288853, 0.044487230479717255, 0.005684166215360165, -0.012234692461788654, -0.011664162389934063, -0.008966472931206226, 0.025779489427804947, -0.04474079981446266, -0.003775357035920024, 0.024413036182522774, -0.015495868399739265, 0.008459335193037987, 0.01046675443649292, -0.00298471562564373, 0.005930691491812468, 0.005560903809964657, -0.019933322444558144, 0.007283058017492294, 0.00830437708646059, -0.0007725923205725849, -0.013199662789702415, -0.014678813517093658, 0.01987697370350361, 0.0068040951155126095, 0.03268219530582428, -0.013079921714961529, -0.0427122488617897, 0.02327197603881359, 0.017482157796621323, 0.03758452460169792, -0.00786767527461052, -0.015988918021321297, 0.022581705823540688, -0.004652282223105431, -0.005430597346276045, 0.0022011175751686096, 0.01694684475660324, -7.984554395079613e-05, 0.013396882452070713, -0.031498875468969345, 0.019440270960330963, 0.01859504170715809, -0.002799821551889181, -0.011466942727565765, 0.04220511019229889, -0.04043012857437134, 0.02955484762787819, 0.01248121727257967, 0.009212997741997242, -0.0063920458778738976, 0.009079170413315296, 0.012939049862325191, -0.03121713176369667, -0.015833960846066475, -0.008367768488824368, 0.02086307294666767, 0.011100676842033863, 0.010840063914656639, -0.032175056636333466, -0.03555597364902496, -0.009065083228051662, 0.016538318246603012, -0.011995211243629456, -0.005589078180491924, 0.027343163266777992, 0.0020197455305606127, 0.04820623621344566, -0.01627066172659397, -0.011466942727565765, 0.019496619701385498, 0.0028086260426789522, -0.00640965485945344, -0.014706987887620926, -0.021243426948785782, 0.017566679045557976, 0.004465627484023571, -0.017792074009776115, -0.014425245113670826, 0.020158715546131134, -0.0022064002696424723, 0.018538692966103554, 0.021694215014576912, 0.012805222533643246, -0.013544797897338867, -0.008966472931206226, 0.010607626289129257, -0.02160969190299511, 0.008881949819624424, 0.01018501166254282, -0.027357250452041626, -0.02586401253938675, -0.009846920147538185, 0.013086965307593346, 0.00564894825220108, 0.03023103065788746, 0.011833208613097668, -0.01217834372073412, -0.0009500024025328457, 0.002484621712937951, -0.02887866459786892, -0.019214875996112823, 0.014277329668402672, -0.0019493098370730877, 0.010473798029124737, -0.009846920147538185, -0.034203607589006424, -0.018242863938212395, -0.011192243546247482, 0.014171675778925419, -0.01618613861501217, 0.024314425885677338, 0.014206893742084503, 0.017594853416085243, -0.007367581129074097, 0.0011797990882769227, -0.01504507940262556, 0.02021506428718567, -0.01679188571870327, -0.02383546344935894, -0.007797239348292351, 0.04412096366286278, 0.040683697909116745, 0.011058415286242962, -0.019595229998230934, 0.01875000074505806, -0.012903831899166107, 0.008107156492769718, -0.02444121055305004, 0.033414725214242935, -0.00034557547769509256, -0.006349784322082996, 0.006709006614983082, 0.008198723196983337, -0.0076563674956560135, 0.016143877059221268, -0.014199850149452686, -0.0024951868690550327, 0.022412659600377083, -0.003719008294865489, -0.03628850728273392, -0.0020514416974037886, -0.04127535969018936, -0.02435668744146824, 0.012424868531525135, 0.0015363801503553987, -0.0160734411329031, -0.02455390803515911, -0.020609505474567413, 0.015059166587889194, -0.005888429936021566, -0.01259391475468874, 0.010220229625701904, -0.001664925366640091, -0.01976427622139454, 0.0008034079219214618, -0.006075084675103426, -0.023919986560940742, 0.012974267825484276, -0.03964124992489815, -0.03144252672791481, -0.007261927239596844, -0.007994459010660648, 0.012121994979679585, 0.009938486851751804, -0.024384861811995506, 0.03166792169213295, -0.037218257784843445, -0.034738920629024506, -0.007924024015665054, -0.005825037602335215, 0.0003825542517006397, 0.01753850467503071, -0.007416886277496815, 0.011854339390993118, 0.03468257188796997, -0.00937500037252903, 0.0155522171407938, 0.016636928543448448, -0.034090910106897354, 0.04668482393026352, -0.006628005299717188, 0.014073065482079983, 0.010966848582029343, 0.012699568644165993, -0.015129602514207363, 0.012699568644165993, 0.006149042397737503, -0.0011982884025201201, 0.009797614999115467, 0.019552968442440033, 0.03600676357746124, 0.026666980236768723, 0.016876408830285072, -0.012340346351265907, 0.007494365330785513, -0.006001127418130636, -0.008501596748828888, -0.009184823371469975, 0.009022821672260761, 0.008008546195924282, 0.031245306134223938, -0.006642092484980822, -0.016059353947639465, 0.004071187227964401, 0.014206893742084503, 0.015805786475539207, -0.013763148337602615, 0.0035728542134165764, -0.014037848450243473, -0.022609880194067955, -0.0037013995461165905, 0.005913082510232925, 0.04668482393026352, -0.005994083825498819, -0.014270286075770855, -0.001618261681869626, 0.018961308524012566, 0.03276671841740608, 0.016848234459757805, -0.013650450855493546, -0.021975958719849586, -0.0038986196741461754, 0.013037660159170628, 0.0005652470281347632, 0.0015196516178548336, -0.004740327131003141, 0.0230465829372406, -0.014016717672348022, 0.03020285628736019, -0.009846920147538185, 0.03758452460169792, 0.028625095263123512, 0.03304846212267876, -0.011009110137820244, 0.010368145070970058, 0.002579709980636835, 0.02029958739876747, 0.011044328100979328, -0.0141223706305027, 0.027314988896250725, 0.018327385187149048, -0.009642655961215496, -0.012741830199956894, 0.0009799376130104065, 0.005997605621814728, -0.011678249575197697, 0.005011504516005516, 0.010678062215447426, 0.02658245712518692, 0.012143125757575035, 0.02804752252995968, -0.0005973833613097668, 0.008086025714874268, -0.0077620213851332664, 0.024638431146740913, 0.0006471286178566515, -0.04755822941660881, -0.005786298308521509, 0.009438392706215382, -0.01370679959654808, -0.028216566890478134, -0.025666791945695877, -0.006134955212473869, -0.010121619328856468, 0.02029958739876747, 0.0034548742696642876, 0.023779114708304405, -0.00019766036712098867, 0.011445811949670315, 0.028737792745232582, 0.01807381771504879, -0.031160783022642136, 0.02500469610095024, 0.01717223972082138, -0.0035182663705199957, -0.005152376368641853, -0.030061984434723854, 0.017947033047676086, 0.0077761085703969, 0.00640965485945344, 0.009762397035956383, 0.025807663798332214, 0.004641716834157705, -0.001698382431641221, 0.00803672056645155, 0.026906462386250496, 0.007529583293944597, -0.005053766071796417, 0.015833960846066475, 0.0042226240038871765, -0.012185387313365936, -0.01753850467503071, 0.006797051522880793, 0.0017793835140764713, 0.03087903931736946, 0.014030804857611656, 0.02387772500514984, -0.0391622856259346, -0.00028614531038329005, -0.03361194580793381, 0.015904394909739494, -0.006656179670244455, -0.019707927480340004, 0.004930503666400909, 0.00048512633657082915, -0.0024317947681993246, 0.021750563755631447, -0.003697877749800682, 0.01370679959654808, 0.0010327644413337111, -0.021863261237740517, -0.033640120178461075, -0.025385050103068352, 0.0019387444481253624, -0.012403738684952259, 0.01649605669081211, 0.013361664488911629, -0.0018595041474327445, -0.020510895177721977, -0.02225770242512226, -0.022609880194067955, -0.011607813648879528, 0.004345886874943972, 0.008769252337515354, -0.022384487092494965, 0.009297520853579044, 0.013622277416288853, -0.018327385187149048, 0.009551090188324451, -0.004652282223105431, -0.01735537312924862, -0.021482909098267555, -0.007698629051446915, -0.038204360753297806, -0.011678249575197697, -0.00807898212224245, 0.02048272080719471, 0.020229151472449303, 0.023215627297759056, -0.010030052624642849, -0.01018501166254282, 0.020229151472449303, 0.007304188795387745, -0.021736476570367813, -0.008318464271724224, -0.005920126102864742, 0.017144065350294113, 0.004120492376387119, -0.017524417489767075, -0.010910499840974808, 0.024751128628849983, 0.012453042902052402, 0.024863824248313904, 0.026906462386250496, 0.002889627357944846, 0.0009861007565632463, 0.007938110269606113, -0.011072502471506596, -0.004588889889419079, 0.004624107852578163, 0.026681067422032356, 0.01229104120284319, 0.004965721629559994, -0.018834523856639862, -0.0077761085703969, -0.012129038572311401, -0.008057851344347, 0.0036203982308506966, 0.006930879317224026, -0.02737133763730526, -0.002761082025244832, -0.021806912496685982, -0.010649887844920158, -0.021257514134049416, -0.006709006614983082, -0.006835790816694498, 0.030061984434723854, -0.011445811949670315, 0.001006350968964398, 0.02597671002149582, -0.016735536977648735, 0.0005247464287094772, -0.030061984434723854, -0.0005564425373449922, -0.0031379132997244596, 0.0004283374873921275, -0.028484223410487175, 0.003442547982558608, 0.01634109765291214, 0.010924587026238441, 0.010685105808079243, -0.014277329668402672, -0.0038281837478280067, 0.005719384178519249, -0.012784091755747795, -0.025694966316223145, 0.018200602382421494, 0.0015425432939082384, -0.009382043965160847, 0.00047235985402949154, 0.013079921714961529, 0.020468633621931076, 0.005353118292987347, 0.0380353145301342, -0.008762208744883537, -0.018890872597694397, 0.016129789873957634, -0.025244178250432014, -0.0010759063297882676, 0.013509579934179783, -0.00608917186036706, -0.0037154864985495806, -0.001951070735231042, -0.007797239348292351, 0.01984879933297634, -0.027469947934150696, -0.009741266258060932, 0.0029952810145914555, -0.007124577648937702, -0.005670079030096531, -0.0037788788322359324, -0.020623592659831047, -0.012791135348379612, 0.012917919084429741, -0.007099925074726343, -0.010755541734397411, -0.012826352380216122, -0.03189331293106079, -0.013988543301820755, -0.01724267564713955, -0.010854151099920273, 0.009832832962274551, -0.029752066358923912, -0.010917543433606625, -0.008712904527783394, -0.006529395468533039, 0.010283621959388256, 0.011438768357038498, 0.016665102913975716, 0.017834335565567017, -0.028935011476278305, 0.00018665478273760527, -0.022638054564595222, 0.013636364601552486, 0.017989294603466988, 0.01362932100892067, -0.00867768656462431, -0.008480465970933437, -0.009551090188324451, -0.007726803421974182, 0.012213561683893204, -0.00814941804856062, 0.005782776512205601, -0.026836026459932327, 0.005007982719689608, 0.003870445303618908, 0.017862509936094284, -0.025469573214650154, 0.017383547499775887, -0.027019159868359566, 0.0063180881552398205, 0.03197783604264259, 0.013875845819711685, 0.04231780767440796, 0.01342505682259798, 0.021637866273522377, -0.013995586894452572, -0.014143501408398151, -0.032175056636333466, -0.03535875305533409, -0.0078113265335559845, 0.0076141064055264, 0.02507513202726841, 0.018609128892421722, 0.03817618638277054, -0.026526108384132385, 0.014333678409457207, -0.01097389217466116, 0.005983518436551094, 0.03132982924580574, 0.00012502347817644477, 0.016538318246603012, -0.034738920629024506, 0.0038105749990791082, -0.0008364247041754425, -0.013615233823657036, 0.020060105249285698, 0.014157588593661785, -0.027343163266777992, 0.014368896372616291, 0.0036133546382188797, 0.02399042248725891, -0.008487509563565254, -0.026934636756777763, -0.013171488419175148, -0.00024212294374592602, 0.0020109410397708416, 0.0006841073627583683, -0.0449661910533905, -0.01980653777718544, 0.010339970700442791, 0.02379320189356804, -0.01773572526872158, 0.010361101478338242, 0.014390027150511742, -0.03535875305533409, -0.0010433297138661146, -0.0011481029214337468, 0.011903644539415836, -0.015566304326057434, -0.020468633621931076, -0.021933697164058685, -0.026216192170977592, -0.01395332533866167, -0.007952197454869747, 0.015411345288157463, -0.00536720547825098, -0.004719196353107691, -0.026441587135195732, 0.01867956481873989, -0.015369083732366562, 0.002417707582935691, -0.01248121727257967, -0.0016992628807201982, -0.0022328137420117855, -0.011382419615983963, -0.03355559706687927, -0.012995398603379726, -0.0011384180979803205, 0.015566304326057434, -0.005863777361810207, -0.0013761387672275305, -0.04223328456282616, -0.007004837039858103, 0.02466660551726818, 0.014749249443411827, 0.054094668477773666, 0.001530217006802559, 0.0282024797052145, 0.03152704983949661, -0.007353493943810463, 0.003148478688672185, -0.01410123985260725, -0.0006105900392867625, 0.003535875352099538, 0.019862886518239975, 0.017637114971876144, -0.00010268214100506157, -0.03248497471213341, 0.004507889039814472, -0.004092318005859852, -0.002023267326876521, -0.02021506428718567, 0.024934260174632072, 0.025920361280441284, -0.0008751644054427743, -0.013742017559707165, -0.015185951255261898, -0.03265402093529701, -7.968045974848792e-05, 0.00017388830019626766, -0.010304752737283707, 0.020792637020349503, -0.008987603709101677, -0.03180878981947899, 0.029357627034187317, -0.02473704144358635, 0.00346015696413815, 0.0008003264083527029, -0.01622840017080307, 0.0137983663007617, 0.006268783006817102, -0.013340533711016178, 0.003005846170708537, 0.013530710712075233, 0.024610256776213646, -0.017608940601348877, -0.005898995324969292, 0.022454921156167984, 0.00234903278760612, -0.004310668911784887, -0.010699192993342876, 0.01976427622139454, 0.03504883497953415, 0.0060222577303647995, -0.01362932100892067, 0.018933134153485298, 0.00898056011646986, -0.009107344783842564, -0.0007717118714936078, -0.0030569122172892094, -0.02104620635509491, -0.010438580065965652, 0.0010503733064979315, 0.01581987366080284, 0.021567432209849358, -0.009586307220160961, 0.007832457311451435, -0.012283997610211372, -0.02435668744146824, -0.016017092391848564, -0.0014782706275582314, 0.00434940867125988, -0.002664232859387994, -0.012903831899166107, -0.02552592195570469, -0.0048248497769236565, 0.0013497252948582172, 0.0236100684851408, -0.02007419243454933, 0.03964124992489815, 0.043388430029153824, -0.03735912963747978, 0.006001127418130636, -0.05809541791677475, 0.013199662789702415, -0.035302404314279556, 0.03023103065788746, 0.004194449633359909, 0.0028825837653130293, -0.019834712147712708, 0.009959617629647255, -0.029188580811023712, -0.0241312924772501, 0.01248121727257967, -0.017228588461875916, 0.0022891624830663204, -0.006057475693523884, 0.009593350812792778, 0.017834335565567017, 0.026821939274668694, -0.03487979248166084, -0.007487321738153696, 0.008874906226992607, -0.01366453804075718, 0.031498875468969345, 0.02025732584297657, -0.01029770914465189, 0.004451540298759937, 0.007367581129074097, 0.017594853416085243, 0.03378099203109741, -0.023581894114613533, -0.002100746612995863, -0.00724784005433321, 0.01825695112347603, -0.01886269822716713, 0.01634109765291214, -0.0016120985383167863, -0.033076636493206024, -0.01314331404864788, 0.005458771716803312, -0.0013541275402531028, -0.035330578684806824, 0.025427311658859253, 0.020989857614040375, 0.0029301277827471495, 0.023807289078831673, -0.00830437708646059, -0.011051371693611145, -0.01791885867714882, -0.014650639146566391, -0.018623216077685356, 0.0023912943433970213, 0.0048001972027122974, 0.03499248996376991, 0.01818651519715786, -0.004930503666400909, -0.018890872597694397, 0.005582034587860107, -0.03121713176369667, -0.004268407355993986, -0.010086401365697384, 0.02334241196513176, 0.008910124190151691, -0.00562781747430563, -0.026779677718877792, 0.008184636011719704, 0.003528831759467721, 0.02021506428718567, -0.02282118797302246, 0.0014245633501559496, 0.012375564314424992, -0.013277141377329826, -0.007402799092233181, 0.012051559053361416, -0.037669047713279724, -0.01890495978295803, 0.01057945191860199, 0.009191866964101791, 0.03806348890066147, 0.03623215854167938, 0.007691585458815098, 0.004159232135862112, 0.0018647868419066072, -0.01728493720293045, 0.003902141470462084, -0.007902893237769604, 0.0021359645761549473, -0.02932945266366005, -0.02435668744146824, -0.001332996878772974, -0.003947924822568893, 0.02199004590511322, -0.015791699290275574, 0.003736617276445031, 0.0047086309641599655, 0.0031625658739358187, -0.030033810064196587, -0.00346015696413815, -0.014523854479193687, -0.013601146638393402, 0.013467318378388882, -0.009522915817797184, 0.002331423806026578, 0.04530428349971771, 0.020454546436667442, 0.03242862597107887, -0.009621525183320045, 0.025652704760432243, -0.002220487454906106, 0.007089359685778618, -0.0017089477041736245, -0.007177404593676329, 0.0017961120465770364, -0.013256010599434376, 0.022905711084604263, -0.018876785412430763, 0.016355184838175774, -0.0016032940475270152, -0.004293059930205345, -0.024497559294104576, 0.02399042248725891, -0.0037330954801291227, -0.017299024388194084, 0.013157401233911514, 0.010889369063079357, -0.014876033179461956, -0.007180926389992237, -0.017552591860294342, 0.0008258593734353781, -0.01327009778469801, -0.0003898179274983704, 0.004338843282312155, 0.006469524931162596, -0.018468257039785385, 0.01071328017860651, 0.02029958739876747, -0.0344853512942791, -0.007346450351178646, 0.17062360048294067, -0.01044562365859747, 0.02561044506728649, 0.048093538731336594, -0.014988730661571026, 0.010889369063079357, -0.0018683086382225156, -0.0137983663007617, -0.03248497471213341, 0.02063767984509468, -0.003234762465581298, 0.0008315822342410684, -0.008205766789615154, -0.004229667596518993, 0.0003068358055315912, 0.03000563569366932, -0.011988167650997639, -0.01908809319138527, 0.01368566881865263, -0.0033087199553847313, -0.007649323903024197, 0.005191115662455559, 0.0026149277109652758, -0.03558414801955223, 0.020003756508231163, -0.015101428143680096, 0.006677310448139906, 0.02093350887298584, 0.020454546436667442, 0.029132232069969177, -0.0036591379903256893, -0.009008734486997128, -0.0048988074995577335, -0.0003799129044637084, 0.007445060182362795, -0.021778738126158714, -0.026610631495714188, 0.02093350887298584, 0.017749812453985214, 0.020454546436667442, 0.013594103045761585, -0.041331708431243896, 0.019750189036130905, -0.013072878122329712, -0.03338655084371567, -0.01086119469255209, -0.01920078881084919, -0.008381855674088001, -0.0016860561445355415, -0.00018489388457965106, -0.03383734077215195, -0.016467882320284843, 0.028935011476278305, 0.04764275252819061, -0.004525498021394014, -0.0032470887526869774, 0.021666040644049644, 0.0014562594005838037, 0.021905522793531418, 0.007353493943810463, -0.0052298554219305515, 0.03355559706687927, 0.00210779020562768, 0.03054094687104225, 0.0024300338700413704, 0.0015504673356190324, -0.012917919084429741, -0.014284373261034489, 0.0190176572650671, 0.008924211375415325, -0.018172428011894226, 0.016594666987657547, 0.01108658965677023, 0.0022486618254333735, -0.012037471868097782, -0.031386177986860275, 0.03056912124156952, -0.005539773032069206, 0.03009015880525112, 0.0167214497923851, -0.01976427622139454, -0.004307147115468979, 0.012847483158111572, -0.006029301322996616, -0.006620961707085371, -0.029611196368932724, 0.027244554832577705, 0.011664162389934063, -0.022764839231967926, 0.010431536473333836, -0.0007470593554899096, -0.014087152667343616, -0.0024071421939879656, 0.0005828559515066445, -0.00640965485945344, 0.014509767293930054, -0.012734786607325077, 0.03056912124156952, -0.01927122473716736, 0.006001127418130636, -0.038993239402770996, -0.004152188543230295, 0.013284184969961643, -0.011044328100979328, 0.01362932100892067, -0.0023666417691856623, 0.023328324779868126, -0.009558133780956268, 0.013037660159170628, -0.03783809393644333, -0.009804658591747284, -0.0270332470536232, -0.007740890607237816, -0.0011718750465661287, 0.006244130432605743, 0.013467318378388882, 0.007131621241569519, -0.017651202157139778, 0.008726991713047028, -0.006092693656682968, -0.0057405149564146996, -0.042261458933353424, -0.02025732584297657, -0.010833020322024822, -0.004039491061121225, 0.012544609606266022, -0.04657212644815445, 0.011530335061252117, 0.016087528318166733, 0.010255447588860989, 0.0030357814393937588, -0.04195154085755348, -0.011621900834143162, 0.005289725959300995, -0.0009112627012655139, -0.0013708560727536678, -0.0007466191309504211, -0.02606123313307762, 0.008269159123301506, 0.008346637710928917, 0.004775545094162226, 0.014692900702357292, 0.033414725214242935, -0.024342600256204605, -0.01391106378287077, -0.013847671449184418, 0.00453606341034174, -0.005560903809964657, -0.03716190904378891, -0.01841190829873085, -0.004810762591660023, 0.0021130729001015425, -0.008297333493828773, -0.02594853565096855, 0.005927169695496559, -0.009614481590688229, -0.000492169929202646, -0.02887866459786892, 0.0052052028477191925, 0.025990797206759453, -0.023398760706186295, -0.0009500024025328457, -0.018059730529785156, 0.002236335538327694, -0.005324943922460079, -0.014735162258148193, -0.18054094910621643, 0.03163974732160568, 0.018200602382421494, -0.026554282754659653, 0.0073746247217059135, -3.389721314306371e-05, 0.014340722002089024, 0.0005247464287094772, -0.017989294603466988, -0.020285500213503838, 0.035189706832170486, -0.03840157762169838, -0.0334428995847702, -0.011452855542302132, 0.005624295677989721, -0.006275826599448919, -0.007480278145521879, 0.007966284640133381, 0.009332738816738129, 0.019003570079803467, 0.03200601041316986, -0.03169609606266022, 0.0006788246682845056, 0.00886786263436079, -0.027019159868359566, 0.011304940097033978, 0.010431536473333836, 0.021454734727740288, -0.017186326906085014, -0.016763711348176003, 0.022722577676177025, 0.009853963740170002, 0.03817618638277054, 0.024835649877786636, 0.01060058269649744, 0.02552592195570469, -0.0167214497923851, -0.021806912496685982, -0.005603164900094271, 0.027836214751005173, 0.0028438440058380365, 0.0070294891484081745, -0.01743989624083042, 0.014411157928407192, 0.012509391643106937, 0.013720886781811714, 0.01073441095650196, -0.005765167530626059, 0.001785546657629311, -0.0011551465140655637, 0.028287002816796303, -0.029583021998405457, -0.007536626886576414, 0.008628381416201591, 0.024610256776213646, 0.02248309552669525, -0.0024300338700413704, 0.005874342750757933, -0.006937922909855843, -0.025723140686750412, -0.01604526676237583, -0.017989294603466988, -0.0007950437138788402, -0.00721966614946723, -0.03290759027004242, -0.003268219530582428, -0.024567995220422745, 0.008987603709101677, -0.040120210498571396, 0.015073253773152828, -0.00458536809310317, 0.019750189036130905, 0.012044515460729599, 0.03634485602378845, -0.0025339266285300255, -0.007117534056305885, 0.003372112289071083, 0.0369083397090435, 0.008572032675147057, 0.0027505166362971067, -0.020116453990340233, 0.04922051355242729, -0.02187734842300415, 0.0010371666867285967, 0.03023103065788746, -0.0011586683103814721, 0.024708867073059082, 0.004779066890478134, -0.004451540298759937, -0.005729949567466974, 0.015566304326057434, -0.02044045925140381, -0.006124389823526144, -0.008917167782783508, -0.01511551532894373, 0.011995211243629456, -0.008600207045674324, 0.0026712764520198107, 0.013326446525752544, -0.01751033030450344, -0.005934213288128376, -0.012889744713902473, -0.009706048294901848, -0.01674962416291237, 0.002134203677996993, 0.028188392519950867, -0.02921675518155098, 0.028906837105751038, 0.03392186388373375, 0.020384110510349274, 0.005127723794430494, 0.005814472213387489, -0.0037401390727609396, 0.014136457815766335, 0.006314566358923912, 0.005976474843919277, 0.005018548108637333, -0.01449568010866642, 0.01674962416291237, 0.02477930299937725, 0.009445436298847198, -0.01219947449862957, -0.0012810504995286465, 0.014706987887620926, 0.004131057765334845, 0.010537190362811089, -0.10013148188591003, -0.04724831134080887, 0.03772539645433426, 0.010755541734397411, 0.0018524606712162495, 0.028850490227341652, -0.013692712411284447, 0.03716190904378891, -0.02963937073945999, -0.0019633970223367214, -0.010516059584915638, -0.013094008900225163, 0.002132442779839039, -0.0029952810145914555, 0.007438016589730978, -0.0008104515145532787, 0.005458771716803312, 0.00967787392437458, -0.014932381920516491, 0.019595229998230934, -0.029498498886823654, -0.0016710885101929307, -0.006515308283269405, 0.011149981990456581, 0.004152188543230295, 0.02725864201784134, -0.02424398995935917, 0.028906837105751038, 0.0018841567216441035, 0.011403550393879414, 0.018623216077685356, -0.0019950929563492537, 0.01303061656653881, -0.019482532516121864, -0.013594103045761585, 0.0009790571639314294, -0.015495868399739265, -0.02221544086933136, 0.044149138033390045, -0.00963561236858368, 0.009234128519892693, -0.0033333725295960903, -0.002949497662484646, -0.017947033047676086, -0.016848234459757805, -0.022159092128276825, -0.035865891724824905, 0.023624155670404434, -0.007980371825397015, 0.006807616911828518, -0.020722202956676483, 0.010706236585974693, 0.000927991233766079, -0.0004230548220220953, -0.00950882863253355, 0.018172428011894226, 0.03324568271636963, -0.006694919429719448, -0.022074569016695023, 0.02439894899725914, -0.014375939965248108, -0.0008443487458862364, -0.009445436298847198, 0.0167214497923851, 0.002933649579063058, -0.004926981870085001, -0.02530052699148655, -0.024483472108840942, 0.040289256721735, -0.019891060888767242, -0.01589030772447586, 0.01261504553258419, -0.01069214940071106, 0.017975207418203354, -0.02966754510998726, 0.003902141470462084, -0.00126168061979115, -0.014009674079716206, 0.012382607907056808, -0.02545548602938652, -0.014650639146566391, 0.011635988019406796, -0.018017468973994255, -0.0201023668050766, 0.0323159284889698, 0.011311983689665794, 0.0012669633142650127, 0.004187406040728092, 0.004870633129030466, -0.01927122473716736, 0.02153925783932209, -0.005339031107723713, 0.03719008341431618, -0.026906462386250496, -0.006360349711030722, 0.00495163444429636, 0.004120492376387119, 0.027427686378359795, -0.019595229998230934, 0.008529771119356155, 0.013608190231025219, -0.005155898164957762, -0.054799024015665054, 0.004229667596518993, -0.012741830199956894, -0.011051371693611145, -0.006990749854594469, -0.016355184838175774, 0.014805598184466362, -0.029075883328914642, 0.01649605669081211, 0.013072878122329712, -0.015200038440525532, 0.004803718999028206, -0.006987228058278561, 0.008290289901196957, -0.021285688504576683, -0.010868238285183907, 0.016763711348176003, -0.008748122490942478, 0.020158715546131134, -0.002306771231815219, 0.010607626289129257, -0.02383546344935894, 0.009537003003060818, 0.016369272023439407, -0.00022517434263136238, 0.027695342898368835, 0.001522292965091765, 0.038457926362752914, -0.0014703465858474374, -0.02025732584297657, 0.02617393061518669, -0.02228587679564953, 0.0035393971484154463, 0.014220980927348137, 0.016242487356066704, -0.00382114015519619, -0.00234903278760612, 0.019820624962449074, 0.02804752252995968, -0.02160969190299511, -0.008297333493828773, -0.038570623844861984, 0.0014571398496627808, -0.005240420810878277, -0.01890495978295803, 0.0021465299651026726, -0.007677498273551464, 0.007388711906969547, -0.008226897567510605, 0.003711964935064316, 0.004113448783755302, 0.011142938397824764, -0.023018408566713333, -0.001570717548020184, -0.014242111705243587, -0.008945342153310776, -0.003123826114460826, -0.0033192853443324566, -0.003243566956371069, -0.016763711348176003, 0.03741547837853432, -0.004116970580071211, 0.007473234552890062, -0.03279489278793335, -0.013016529381275177, -0.010903456248342991, -0.014735162258148193, -0.016467882320284843, 0.005282682366669178, -0.009861007332801819, -0.01393219456076622, -0.03516153246164322, 0.021060293540358543, 0.015059166587889194, -0.006473046727478504, -0.013277141377329826, -0.0015733588952571154, 0.020060105249285698, 0.0006964336498640478, 0.022342225536704063, 0.055785126984119415, -0.02228587679564953, -0.032625846564769745, -0.0033985257614403963, 0.04586777091026306, -0.011924775317311287, 0.0008443487458862364, 0.011459899134933949, 0.014044891111552715, -0.03023103065788746, -0.03873967006802559, 0.01822877675294876, 0.01728493720293045, -0.0006929118535481393, 0.03304846212267876, -0.0014905969146639109, -0.027512209489941597, 0.011114764027297497, -0.01449568010866642, 0.030287379398941994, -0.0056665572337806225, -0.00019942126527894288, -0.012453042902052402, -0.020355936139822006, -0.019975582137703896, -0.019834712147712708, -0.007346450351178646, -0.004102883394807577, -0.0311044342815876, 0.017932945862412453, 0.024455297738313675, -0.0026994505897164345, -0.014904207549989223, 0.003595745889469981, -0.01743989624083042, 0.01724267564713955, -0.010339970700442791, 0.009713091887533665, -0.024145379662513733, 0.020905334502458572, 0.022342225536704063, 0.011037284508347511, 0.005155898164957762, 0.002380728954449296, 0.03197783604264259, 0.03707738593220711, 0.0044128005392849445, -0.0152282128110528, -0.01178390346467495, 0.01927122473716736, 0.00527916057035327, -0.007318275980651379, -0.027962999418377876, -0.013812453486025333, -0.03056912124156952, -0.028258828446269035, -0.020426372066140175, -0.012164256535470486, 0.0035499625373631716, 0.08114200085401535, -0.005983518436551094, -0.016059353947639465, 0.007733847014605999, 0.00895942933857441, 0.014876033179461956, 0.0069203139282763, 0.011509204283356667, -0.027991173788905144, -0.005652470048516989, 0.02639932557940483, -0.013530710712075233, 0.0039831423200666904, -0.04533245787024498, 0.010227273218333721, 0.0064836121164262295, -0.013763148337602615, -0.002887866459786892, -0.013094008900225163, -0.014636551961302757, 0.003197783837094903, 0.013763148337602615, -0.006385002285242081, 0.004004273097962141, 0.007790195755660534, 0.007853588089346886, -0.012114951387047768, 0.0024159466847777367, -0.03425995633006096, -0.020172802731394768, -0.017299024388194084, 0.016425620764493942, -0.026145756244659424, -0.02146882191300392, -0.02523009106516838, 0.005212246440351009, -0.017651202157139778, -0.006525873672217131, 0.031132608652114868, -0.010361101478338242, -0.007740890607237816, 0.0702667161822319, -0.022018220275640488, -0.017594853416085243, 0.01272774301469326, -0.012298084795475006, 0.01969384029507637, -0.009713091887533665, -0.04355747625231743]

In [97]:
def search(query):
    search_params = {
        "metric_type": "L2"
    }

    results = driving_collection.search(
        data=[query],
        anns_field="embedding",
        param=search_params,
        limit=5,
        output_fields=["message"]
    )

    parsed_response = []
    for hit in results[0]:
        row = []
        row.extend([hit.id, hit.score, hit.entity.get('message')])
        parsed_response.append(row)

    return parsed_response

In [98]:
print(search(test_query))

[[446442801595687197, 0.0, 'Stay focused, be aware of surrounding vehicles and maintain speed'], [446442801595687199, 0.007114636246114969, 'Stay alert, approaching vehicles and traffic light ahead'], [446442801595687201, 0.00769690191373229, 'Stay focused, be prepared to slow down for the light'], [446442801595687229, 0.011966307647526264, 'Stay alert, multiple vehicles and a traffic light ahead'], [446442801595687225, 0.014330971986055374, 'Stay alert, multiple vehicles and a traffic light ahead']]
